In [1]:
import pandas as pd
import numpy as np
from utilities import loadPickle, savePickle
from tqdm import tqdm

In [96]:
import os


raw_data = {}
base_path = './labels/'
temp = []
data = []
for session in os.listdir(base_path):
    session_path = base_path + session +'/'
    raw_data[session] = {}
    print(session)
    for file in os.listdir(session_path):
        file_path = session_path + file
        file_name = file.split('-')[1]
        df = pd.read_csv(file_path, encoding='shiftjis', delimiter= '\t', header = 1)
        df['folder'] = session
        df['file']  = file_name        
        #print(df.columns)
        df['start frame']  = df['ti'].mul(30).astype(int)
        df['stop frame'] = df['tf'].mul(30).astype(int)
        raw_data[session][file_name] = df[['folder', 'file','ti', 'tf','start frame', 'stop frame', 'cleantext1', 'label1', 'label2', 'label3']]
        #data.append(raw_data[session][file_name])
        print(file_path)
#data = pd.concat(data)   

20170720
./labels/20170720/20170720-152000-000-drrole2-nu-yko.ctr
./labels/20170720/20170720-152500-000-drrole2-nu-yko.ctr
./labels/20170720/20170720-153000-000-drrole2-nu-yko.ctr
./labels/20170720/20170720-153500-000-drrole2-nu-yko.ctr
./labels/20170720/20170720-154000-000-drrole2-nu-yko.ctr
./labels/20170720/20170720-154500-000-drrole2-nu-yko.ctr
./labels/20170720/20170720-155000-000-drrole2-nu-yko.ctr
./labels/20170720/20170720-155500-000-drrole2-nu-yko.ctr
./labels/20170720/20170720-160000-000-drrole2-nu-yko.ctr
./labels/20170720/20170720-160500-000-drrole2-nu-yko.ctr
20170804
./labels/20170804/20170804-164000-000-drrole-yko-nu.ctr
./labels/20170804/20170804-164500-000-drrole-yko-nu.ctr
./labels/20170804/20170804-165000-000-drrole-yko-nu.ctr
./labels/20170804/20170804-165500-000-drrole-yko-nu.ctr
./labels/20170804/20170804-170000-000-drrole-yko-nu.ctr
./labels/20170804/20170804-170500-000-drrole-yko-nu.ctr
./labels/20170804/20170804-171000-000-drrole-yko-nu.ctr
./labels/20170804/20

## Check to see if the files are complete at the end of all this. I think some label files are missing and should be checked again same for the data file

In [180]:
def fixGaze(x:str):
    x = x.strip().strip('[').strip(']')
    a = x.split()
    a = [float(i) for i in a]
    return np.array(a)

def loadGazeFile(session, file):
    '''
    Load the 5 files and concat them together and return them so they can be used to get the data  for one data frame in raw_data
    '''
    base_path = './data/'
    people = ['video1', 'video2', 'video3']
    gaze_data = []
    for person in people:
        subject_gaze = []
        for i in range(5):
            file_path = f"{base_path}{session}/{person}/{str(int(file)+i*100)}.csv"
            try:
                gaze = pd.read_csv(file_path)
            except Exception as e:
                return np.array([-1, -1])
            gaze['Gaze'] = gaze['Gaze'].apply(fixGaze)  
            gaze = np.vstack(gaze['Gaze'].copy())
            subject_gaze.append(gaze)
        gaze_data.append(np.vstack(subject_gaze))
    gaze_data = np.array(gaze_data)
    return gaze_data

def loadPoseFile(session, file):
    people = ['video1', 'video2', 'video3']
    base_path = './data'

    pose_data = []
    for person in people:
        subject_pose = []
        for i in range(5):
            file_path = f"{base_path}/{session}/Pose/{person}/{str(int(file)+i*100)}.npy"
            try:
                pose = np.load(file_path)
            except:
                return np.array([-1, -1])
            subject_pose.append(pose)
        pose_data.append(np.vstack(subject_pose))
    return np.array(pose_data)

def getGazeData(session, file, start_fram, end_frame):
    people = ['video1', 'video2', 'video3']
    base_path = './data/'
    gaze_data = []
    
    try:
        for person in people:
            subject_gaze = []
            for i in range(5):
                #### this code is slow as hell
                ## It opens and closes 5 files and performs array operations for every single line of the dataframe which is freakin stupid
                file_path = f"{base_path}{session}/{person}/{str(int(file)+i*100)}.csv"
                gaze = pd.read_csv(file_path)
                gaze['Gaze'] = gaze['Gaze'].apply(fixGaze)
                gaze = np.vstack(gaze['Gaze'])
                subject_gaze.append(gaze)
            #print(np.vstack(subject_gaze).shape)
            gaze_data.append(np.vstack(subject_gaze)[start_fram:end_frame])
            #print(gaze_data[0].shape)
        gaze_data = np.array(gaze_data)
        #print(gaze_data.shape)
        #print(gaze_data[0].shape, gaze_data[1].shape, gaze_data[2].shape)
        return gaze_data
    except Exception as e:
        print(e)
        return np.zeros((1,3))

def getPoseData(session, file, start_frame, end):
    people = ['video1', 'video2', 'video3']
    base_path = './data'

    pose_data = []
    try:
        for person in people:
            subject_pose = []
            for i in range(5):
                file_path = f"{base_path}/{session}/Pose/{person}/{str(int(file)+i*100)}.npy"
                pose = np.load(file_path)
                subject_pose.append(pose)
            pose_data.append(np.vstack(subject_pose)[start_frame:end])
        return np.array(pose_data)
    except Exception as e:
        print(e)
        return np.zeros((1,3))

In [162]:
def getGazeTurn(file, start1, stop1, start2, stop2, label1, label2):
    first = np.array([])
    second = np.array([])
    if start1 != -1 and stop1 != -1:
        first = file[:,start1:stop1, :]
    if start2 !=-1 and stop2 != -1:
        second = file[:, start2:stop2, :]
    return (first, label1), (second, label2)

def label2vec(label):
    if type(label)!=str:
        return np.array([-1])
    if 'MS' in label:
        return np.array([1, 0, 0])
    elif 'ML' in label:
        return np.array([0, 1, 0])
    elif 'SL' in label:
        return np.array([0, 0, 1])
    else:
        return  np.array([-1])
    
def label2OneHot(labels):
    r1 = label2vec(labels[0])
    r2 = label2vec(labels[1])
    r3 = label2vec(labels[2])
    if r1.shape[0] == 1 or r2.shape[0] == 1 or r3.shape[0] == 1:
        return np.nan
    return np.hstack([r1,r2,r3])

In [186]:

import math

def cart2sph(x,y,z):
    #print(x, y, z)
    phi = math.asin(y)
    theta = x/math.cos(phi)
    return np.array([theta, phi])

def cart2sphGaze(gaze:np.array):
    cart_gaze = []
    for person in gaze:
        g = []
        for frame in person:
            g.append(cart2sph(frame[0], frame[1], frame[2]))
        cart_gaze.append(g)
    return np.array(cart_gaze)

def cart2sphP(x,y,z):
    XsqPlusYsq = x**2 + y**2
    r = math.sqrt(XsqPlusYsq + z**2)               # r
    elev = math.atan2(z,math.sqrt(XsqPlusYsq))     # theta
    az = math.atan2(y,x)                           # phi
    return (elev, az)

def cart2sphPose(pose:np.array):
    cart_pose = []
    for person in pose:
        p = []
        for frame in person:
            j = []
            for joint in frame:
                j.append(cart2sphP(joint[0], joint[1], joint[2]))
            p.append(j)
        cart_pose.append(p)
    return np.array(cart_pose)            
    
def getStartStopFrame1(ti, tf, window, future, fps):
    if tf-ti < window  +future:
        return (-1, -1)
    start_time = ti + np.random.rand()*((tf-(window+future)) - ti)#could be any random value between start, stop - (future + 0.1)
    start_frame = int(start_time*fps)
    stop_frame = start_frame + int(fps*window)
    return start_frame, stop_frame
def getStartStopFrame2(ti, tf, window, future, fps):
    start_frame = int(fps*(tf-(future + window) + np.random.rand()*(future)))#int(fps*window)
    stop_frame = start_frame + int(fps*window)
    return start_frame, stop_frame


def makeDataset2(raw_data, window, future, fps):
    turns = []
    for session in tqdm(raw_data.keys()):
        for file in raw_data[session].keys():
            df = raw_data[session][file][['folder', 'file', 'ti', 'tf', 'label1', 'label2', 'label3']].copy()
            #print(df.columns)
            gaze_file = loadGazeFile(session, file)
            pose_file = loadPoseFile(session, file)
            if len(gaze_file.shape) !=3 or len(pose_file.shape) != 4:
                print('___________________________________')
                continue
            df[['start1', 'stop1']] = df.apply(lambda row: getStartStopFrame1(row['ti'], row['tf'], window, future, fps), axis=1, result_type='expand')
            df[['start2', 'stop2']] = df.apply(lambda row: getStartStopFrame2(row['ti'], row['tf'], window, future, fps), axis=1, result_type='expand')
            
            df['Encoded CR'] = df.apply(lambda row:label2OneHot([row['label1'], row['label2'], row['label3']]), axis=1)
            df['Encoded FR'] = df['Encoded CR'].shift(-1)
            
            #df.dropna(inplace=True)
            #print(df.head())
            for i, row in df.iterrows():
                #print(row)
                cr = row['Encoded CR']
                fr = row['Encoded FR']
                start1, stop1, start2, stop2  = row['start1':'stop2']
                gaze = gaze_file[:, start1: stop1]
                pose = pose_file[:, start1: stop1]
                turns.append([gaze, pose, cr, cr])
                if i!= len(df):
                    gaze = gaze_file[:, start2: stop2]
                    #print(gaze.shape)
                    pose = pose_file[:, start2: stop2]
                    turns.append([gaze, pose, cr, fr])  

    return turns
a= makeDataset2(raw_data, 2, 1, 30)

  0%|          | 0/19 [00:00<?, ?it/s]C:\Users\irdal\AppData\Local\Temp\ipykernel_30900\2146178411.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  gaze_data = np.array(gaze_data)
C:\Users\irdal\AppData\Local\Temp\ipykernel_30900\2146178411.py:44: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(pose_data)
C:\Users\irdal\AppData\Local\Temp\ipykernel_30900\2146178411.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shap

___________________________________
___________________________________


C:\Users\irdal\AppData\Local\Temp\ipykernel_30900\2146178411.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  gaze_data = np.array(gaze_data)
C:\Users\irdal\AppData\Local\Temp\ipykernel_30900\2146178411.py:44: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(pose_data)
C:\Users\irdal\AppData\Local\Temp\ipykernel_30900\2146178411.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do 

___________________________________
___________________________________


C:\Users\irdal\AppData\Local\Temp\ipykernel_30900\2146178411.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  gaze_data = np.array(gaze_data)
C:\Users\irdal\AppData\Local\Temp\ipykernel_30900\2146178411.py:44: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(pose_data)
 11%|█         | 2/19 [00:03<00:28,  1.66s/it]

...

___________________________________


C:\Users\irdal\AppData\Local\Temp\ipykernel_30900\2146178411.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  gaze_data = np.array(gaze_data)
C:\Users\irdal\AppData\Local\Temp\ipykernel_30900\2146178411.py:44: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(pose_data)


___________________________________


C:\Users\irdal\AppData\Local\Temp\ipykernel_30900\2146178411.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  gaze_data = np.array(gaze_data)
C:\Users\irdal\AppData\Local\Temp\ipykernel_30900\2146178411.py:44: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(pose_data)
100%|██████████| 19/19 [00:48<00:00,  2.53s/it]

___________________________________


In [211]:
len(df['Future Roles'].dropna())

5184

In [194]:
df = pd.DataFrame(columns=['Gaze', 'Pose', 'Current Role', 'Future Roles'], data=a)

In [215]:
y = df.dropna()

In [217]:
import pickle


with open('abc.pth.tar', 'wb') as handle:
    pickle.dump(df, handle)

In [105]:

#getStartStopFrame1(x['ti'], x['tf'], 2, 2, 30))

,folder,file,ti,tf,start frame,stop frame,cleantext1,label1,label2,label3,a,b,start1,stop1,start2,stop2
0,20170720,152000,247.9659,254.3827,7438,7631,話題を何にするか,AL,AL,MS,7475,7505,7511,7541,7582,7612
1,20170720,152000,254.3827,258.8760,7631,7766,NaN,MS,AL,AL,7665,7695,7671,7701,7709,7739
2,20170720,152000,258.8760,261.7606,7766,7852,浴衣を着る予定,AL,MS,AL,7775,7805,7784,7814,7794,7824
3,20170720,152000,261.7606,265.0535,7852,7951,NaN,AL,AL,MS,7886,7916,7854,7884,7916,7946
4,20170720,152000,265.0535,265.9894,7951,7979,NaN,MS,SL,ML,-1,-1,-1,-1,7919,7949


In [137]:
y = y[['folder', 'file', 'label1', 'label2', 'label3', 'Encoded FR', 'Encoded CR']]
z = y.dropna()

In [138]:
temp = []
z.apply(lambda row:temp.append(getGazeTurn(gaze_file, row['start1'], row['stop1'], row['start2'], row['stop2'])), axis=1)

KeyError: 'start1'

In [119]:
temp[5][1].shape

(3, 30, 3)

In [182]:
def fliparray(arr, con= 1):
    if con == 1:
        temp = arr.copy()
        arr[0] = temp[1]
        arr[1] = temp[2]
        arr[2] = temp[0]
        return arr.copy()
    else:
        temp = arr.copy()
        arr[0] = temp[2]
        arr[1] = temp[0]
        arr[2] = temp[1]
        return arr.copy()
    

def getStartStopFrame1(ti, tf, window, future, fps):
    if tf-ti < window  +future:
        return (-1, -1)
    start_time = ti + np.random.rand()*((tf-(window+future)) - ti)#could be any random value between start, stop - (future + 0.1)
    start_frame = int(start_time*fps)
    stop_frame = start_frame + int(fps*window)
    return (start_frame, stop_frame)
def getStartStopFrame2(ti, tf, window, future, fps):
    start_frame = int(fps*(tf-(future + window) + np.random.rand()*(future)))#int(fps*window)
    stop_frame = start_frame + int(fps*window)
    return (start_frame, stop_frame)

def makeData(raw_data, window = 2, future_time = 1, fps = 30):
    turns = []
    window_fps = + int(fps*window)
    for session in tqdm(raw_data.keys()):
        print(session)
        for file in (raw_data[session].keys()):
            print(file)
            df = raw_data[session][file]

            for i, _ in df.iterrows():
                tf = df.loc[i]["tf"]
                ti = df.loc[i]["ti"]
                if tf-ti < window + future_time:
                    continue
                start_frame, stop_frame = getStartStopFrame1(ti, tf, window, future_time, fps)
                if start_frame == -1:
                    continue
                r1, r2, r3 = df.iloc[i]['label1':'label3']
                gaze_data = getGazeData(session, file, start_frame, stop_frame)
                pose_data = getPoseData(session, file, start_frame, stop_frame)
                try:
                    if gaze_data.shape[1] != window_fps or pose_data.shape[1]!=window_fps:
                        break
                except Exception as e:
                    break
                #print(1, file, pose_data.shape, gaze_data.shape)
                turns.append([session, file, start_frame, stop_frame, gaze_data.copy(), pose_data.copy(), r1, r2, r3, r1,r2, r3])
                turns.append([session, file, start_frame, stop_frame, fliparray(gaze_data, 1), fliparray(pose_data, 1), r2, r3, r1, r2,r3, r1])
                turns.append([session, file, start_frame, stop_frame, fliparray(gaze_data, 2), fliparray(pose_data, 1), r3, r1, r2, r3,r1, r2])

                if i == len(df) -1:
                    continue
                gaze_data = getGazeData(session, file, start_frame, stop_frame)
                pose_data = getPoseData(session, file, start_frame, stop_frame)
                #print(2, file, pose_data.shape, gaze_data.shape)
                nr1, nr2, nr3 = df.iloc[i+1]['label1':'label3']
                start_frame, stop_frame = getStartStopFrame2(ti, tf, window, future_time, fps)
                #turns.append([session, file, start_frame, stop_frame, gaze_data, pose_data, r1,r2, r3, nr1, nr2, nr3])
                turns.append([session, file, start_frame, stop_frame, gaze_data.copy(), pose_data.copy(), r1, r2, r3, nr1,nr2, nr3])
                turns.append([session, file, start_frame, stop_frame, fliparray(gaze_data, 1), fliparray(pose_data, 1), r2, r3, r1, nr2,nr3, nr1])
                turns.append([session, file, start_frame, stop_frame, fliparray(gaze_data, 2), fliparray(pose_data, 1), r3, r1, r2, nr3,nr1, nr2])
    return turns


def cLabel2vec(label):
    if 'MS' in label:
        return np.array([1, 0, 0, 0])
    elif 'ML' in label:
        return np.array([0, 1, 0, 0])
    elif 'SL' in label:
        return np.array([0, 0, 1, 0])
    elif 'AL' in label:
        return np.array([0, 0, 0, 1])
    else:
        return  np.array([-1])

def label2vec(label):
    if type(label)!=str:
        return np.array([-1])
    if 'MS' in label:
        return np.array([1, 0, 0])
    elif 'ML' in label:
        return np.array([0, 1, 0])
    elif 'SL' in label:
        return np.array([0, 0, 1])
    else:
        return  np.array([-1])
    
def label2OneHot(labels):
    r1 = label2vec(labels[0])
    r2 = label2vec(labels[1])
    r3 = label2vec(labels[2])
    if r1.shape[0] == 1 or r2.shape[0] == 1 or r3.shape[0] == 1:
        return np.nan
    return np.hstack([r1,r2,r3])



In [183]:
import pickle

def makeDataset(data, window, future, fps=30):
    turns = makeData(data, window, future, fps)
    df = pd.DataFrame(turns)#
    df.columns = ['folder', 'file', 'fi', 'ff', 'Gaze', 'Pose', 'CR1', 'CR2', 'CR3', 'NR1', 'NR2', 'NR3']
    df = df[['Gaze', 'Pose', 'CR1', 'CR2', 'CR3', 'NR1', 'NR2', 'NR3']]
    df['Gaze'] = df['Gaze'].apply(cart2sphGaze)
    #df['Pose'] = df['Pose'].apply(cart2sphPose)
    df['Encoded Future Roles'] = df.apply(lambda x:label2OneHot([x['NR1'], x['NR2'], x['NR3']]), axis=1)
    df['Encoded Current Roles'] = df.apply(lambda x:label2OneHot([x['CR1'], x['CR2'], x['CR3']]), axis=1)
    df = df.dropna().reset_index()
    with open(f'win{window}_f{future}.pth.tar', 'wb') as handle:
        pickle.dump(df, handle)

In [184]:
conditions = [(2,0.5), (2, 1), (2, 1.5), (2,2),
              (1,0.5), (1, 1), (1, 1.5), (1,2),
              (1.5,0.5), (1.5, 1), (1.5, 1.5), (1.5,2),
              (0.5,0.5), (0.5, 1), (0.5, 1.5), (0.5,2)]
for condition in conditions:
    window = condition[0]
    future = condition[1]
    print(window, future)
    makeDataset(raw_data, window, future, 30)

2 0.5


  0%|          | 0/19 [00:00<?, ?it/s]

20170720
152000
152500
153000
153500
154000
154500
155000
155500
160000


C:\Users\irdal\AppData\Local\Temp\ipykernel_28852\2858089190.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  gaze_data = np.array(gaze_data)
C:\Users\irdal\AppData\Local\Temp\ipykernel_28852\2858089190.py:44: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(pose_data)


160500


...

20170829
165500
170000
170500
171000
171500
172000
172500


 21%|██        | 4/19 [17:36<55:25, 221.73s/it]  

173000
[Errno 2] No such file or directory: './data/20170829/video1/173400.csv'
[Errno 2] No such file or directory: './data/20170829/Pose/video1/173400.npy'
20170915
112000
112500


 26%|██▋       | 5/19 [18:21<36:51, 157.95s/it]

20171026
124500
125000
125500


In [68]:
import pickle
import pandas as pd
from utilities import loadPickle
import numpy as np

df = pd.DataFrame(loadPickle('./updated_data.pth.tar'))
df.columns = ['folder', 'file', 'fi', 'ff', 'Gaze', 'Pose', 'CR1', 'CR2', 'CR3', 'NR1', 'NR2', 'NR3']
df = df[['Gaze', 'Pose', 'CR1', 'CR2', 'CR3', 'NR1', 'NR2', 'NR3']]
df.head()

,Gaze,Pose,CR1,CR2,CR3,NR1,NR2,NR3
0,"[[[-0.20838876, -0.25313428, -0.9447206], [-0....","[[[[ 0.45749328 0.43738496 -0.35830131], [ 0....",AL,AL,MS,AL,AL,MS
1,"[[[0.66170615, -0.71367747, -0.22980313], [0.6...","[[[[ 0.41196942 0.62693238 -0.50172019], [ 0....",AL,MS,AL,AL,MS,AL
2,"[[[-0.20838876, -0.25313428, -0.9447206], [-0....","[[[[ 0.15192807 0.15658975 -0.56351638], [ 0....",MS,AL,AL,MS,AL,AL
3,"[[[-0.20838876, -0.25313428, -0.9447206], [-0....","[[[[ 0.15192807 0.15658975 -0.56351638], [ 0....",AL,AL,MS,MS,AL,AL
4,"[[[-0.20838876, -0.25313428, -0.9447206], [-0....","[[[[ 0.45749328 0.43738496 -0.35830131], [ 0....",AL,AL,MS,MS,AL,AL


In [69]:
df['Gaze'] = df['Gaze'].apply(cart2sphGaze)
#df['Pose'] = df['Pose'].apply(cart2sphPose)
df['Encoded Future Roles'] = df.apply(lambda x:label2OneHot([x['NR1'], x['NR2'], x['NR3']]), axis=1)
df['Encoded Current Roles'] = df.apply(lambda x:label2OneHot([x['CR1'], x['CR2'], x['CR3']]), axis=1)
df = df.dropna().reset_index()

In [1]:
df.head()

NameError: name 'df' is not defined

In [52]:
df['Encoded Future Roles'] = df.apply(lambda x:label2OneHot([x['NR1'], x['NR2'], x['NR3']]), axis=1)

In [53]:
df['Encoded Current Roles'] = df.apply(lambda x:label2OneHot([x['CR1'], x['CR2'], x['CR3']]), axis=1)

In [59]:
df = df.dropna().reset_index()

In [60]:
df.head()

,index,Gaze,Pose,CR1,CR2,CR3,NR1,NR2,NR3,Encoded Future Roles,Encoded Current Roles
0,35,"[[[0.14793602, -0.9857124, -0.0805354], [0.155...","[[[[ 0.45213404 0.47358507 -0.46827409], [ 0....",MS,SL,ML,MS,SL,ML,"[1, 0, 0, 0, 0, 1, 0, 1, 0]","[1, 0, 0, 0, 0, 1, 0, 1, 0]"
1,36,"[[[-0.17070144, -0.3902507, -0.9047461], [-0.2...","[[[[ 0.44296473 0.60672528 -0.48984349], [ 0....",SL,ML,MS,SL,ML,MS,"[0, 0, 1, 0, 1, 0, 1, 0, 0]","[0, 0, 1, 0, 1, 0, 1, 0, 0]"
2,37,"[[[0.14793602, -0.9857124, -0.0805354], [0.155...","[[[[-0.03676729 0.20102055 -0.28383818], [-0....",ML,MS,SL,ML,MS,SL,"[0, 1, 0, 1, 0, 0, 0, 0, 1]","[0, 1, 0, 1, 0, 0, 0, 0, 1]"
3,45,"[[[-0.20142308, -0.09219728, -0.9751556], [-0....","[[[[ 0.46577752 0.43357387 -0.35742906], [ 0....",MS,ML,SL,MS,ML,SL,"[1, 0, 0, 0, 1, 0, 0, 0, 1]","[1, 0, 0, 0, 1, 0, 0, 0, 1]"
4,46,"[[[0.29234654, -0.01084012, -0.95625097], [0.2...","[[[[ 0.43147999 0.5550915 -0.43064788], [ 0....",ML,SL,MS,ML,SL,MS,"[0, 1, 0, 0, 0, 1, 1, 0, 0]","[0, 1, 0, 0, 0, 1, 1, 0, 0]"


In [15]:
df['Pose'] = df['Pose'].apply(cart2sphPose)

In [16]:
with open('updated_data_cart.pth.tar', 'wb') as handle:
    pickle.dump(df, handle)

In [17]:
df.iloc[0]['Pose'].shape

(3, 60, 33, 2)

Gaze    [[[-0.21540422726598624, -0.25591868518726935]...
Pose    [[[[-0.51511669  0.7629315 ], [-0.49641776  0....
CR1                                                    AL
CR2                                                    AL
CR3                                                    MS
NR1                                                    AL
NR2                                                    AL
NR3                                                    MS
Name: 0, dtype: object